<a href="https://colab.research.google.com/github/MichaelGelo/GRP2_CEPARCO_IP/blob/main/CEPARCO_IP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Group 2 - Implementing Hyyrö’s Bit Vector Algorithm Using CUDA SIMT**
## **GROUP 2 - S11**

**MEMBERS:**

- Alfred Bastin S. Agustines
- Allan David C. De Leon
- Michael Angelo Depasucat
- Kai Hiori J. Padilla


##Check if CUDA is present

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Mar 17 07:39:14 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   40C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
from google.colab import files

uploaded = files.upload()

for filename, content in uploaded.items():
    with open(filename, "wb") as f:
        f.write(content)

KeyboardInterrupt: 

Upload Files

##C

In [9]:
%%writefile C_hyyro.c
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <stdint.h>
#include <time.h>

#define MAX_LENGTH (1 << 24)

typedef uint64_t bitvector;
                                                                      // Added this
int bit_vector_levenshtein(const char *query, const char *reference, int *zero_indices, int *zero_count) {
    int m = strlen(query);
    int n = strlen(reference);
    if (m > MAX_LENGTH || n > MAX_LENGTH) {
        printf("Error: Strings too long for this implementation!\n");
        return -1;
    }

    bitvector Pv = ~0ULL;
    bitvector Mv = 0;
    bitvector Eq[256] = {0};
    bitvector Ph, Mh, Xv, Xh, Xp;

    for (int i = 0; i < m; i++) {
        Eq[(unsigned char)query[i]] |= (1ULL << i);
    }

    int score = m;
    *zero_count = 0;

    for (int j = 0; j < n; j++) {
        Xv = Eq[(unsigned char)reference[j]] | Mv;
        Xh = ((~Xh & Xv) << 1) & Xp;

        Xh = Xh | ((Xv & Pv) + Pv) ^ Pv | Xv | Mv;
        Ph = Mv | ~(Xh | Pv);
        Mh = Xh & Pv;
        Xp = Xv;

        if (Ph & (1ULL << (m - 1))) score++;
        if (Mh & (1ULL << (m - 1))) score--;

        Xv = (Ph << 1);
        Pv = (Mh << 1) | ~(Xh | Xv);
        Mv = Xh & Xv;

        if (score == 0) {
            zero_indices[*zero_count] = j;
            (*zero_count)++;
        }
    }

    return score;
}

char* read_file_into_string(const char* filename) {
    FILE* file = fopen(filename, "r");
    if (!file) {
        perror("Failed to open file");
        return NULL;
    }


    fseek(file, 0, SEEK_END);
    long file_size = ftell(file);
    fseek(file, 0, SEEK_SET);

    char* buffer = (char*)malloc(file_size + 1);
    if (!buffer) {
        perror("Failed to allocate memory");
        fclose(file);
        return NULL;
    }


    size_t bytes_read = fread(buffer, 1, file_size, file);
    if (bytes_read < file_size) {
        perror("Failed to read the file");
        free(buffer);
        fclose(file);
        return NULL;
    }

    buffer[bytes_read] = '\0';

    fclose(file);
    return buffer;
}


int main() {
    const size_t loope = 10;
    clock_t start, end;
    double elapse = 0.0f;

    const char* query = read_file_into_string("input2.txt");

        // Reference sequences to process
    const char* filenames[] = {"GQ167802.1.txt", "input2.txt"};
    int num_references = sizeof(filenames) / sizeof(filenames[0]);
    double elapsed_times[num_references];
    int distances[num_references];

    // Array to store the reference sequences
    const char* references_input[num_references];
    int reference_lengths[num_references];

    for (int i = 0; i < num_references; i++) {
        references_input[i] = read_file_into_string(filenames[i]);
        if (!references_input[i]) {
            fprintf(stderr, "Error reading file: %s\n", filenames[i]);

            for (int j = 0; j < i; j++) {
                free((void*)references_input[j]);
            }
            return -1;
        }
        reference_lengths[i] = strlen(references_input[i]);

        printf("Contents of file %s:\n", filenames[i]);
        printf("%s\n", references_input[i]);
    }

    printf("Query: %s\n", query);

    for (int i = 0; i < num_references; i++) {
        elapse = 0.0f;
        int distance = 0;

        int first_zero_index = -1;    // ADDED THIS FOR FINDING OUT THE FIRST ZERO
        int *zero_indices = (int*)malloc(strlen(references_input[i]) * sizeof(int));
        int zero_count = 0;

        for (int j = 0; j < loope; j++) {
            start = clock();
            distance = bit_vector_levenshtein(query, references_input[i], zero_indices, &zero_count);
            end = clock();
            elapse += ((double)(end - start)) * 1E3 / CLOCKS_PER_SEC;
        }

        elapsed_times[i] = elapse / loope;
        distances[i] = distance;

        printf("File: %s, Distance: %d, Hits: %d, Zero Indices: ", filenames[i], distances[i], zero_count);
        for (int k = 0; k < zero_count; k++) {
            printf("%d ", zero_indices[k]);
        }
        printf("\nAverage time: %f milliseconds\n", elapsed_times[i]);

        free(zero_indices);
    }

    for (int i = 0; i < num_references; i++) {
        free((void*)references_input[i]);
    }


    return 0;
}


Overwriting C_hyyro.c


In [10]:
%%shell
gcc C_hyyro.c -o C_hyyro

In [11]:
%%shell
./C_hyyro

Contents of file GQ167802.1.txt:
ctttttggcttaggccttggcaccacaattacatttgctagctcccactgacttcttgcctgaataggccttgaaataaatacactagccattattcccctaatagctcaacatcatcacccccgcgctgtcgaagctacaactaaatattttttaacccaagctgctgctgcagccactcttctatttgcaagcattactaacgcctgactaacaggccaatgagaaattcaacaaattacacatcccctcccaaccacaataattacccttgcccttgccctcaaaatcggtcttgccccccttcatgcttgactccccgaagttttacaaggactagatcttaccacaggattaatcctctcaacctgacaaaaacttgctcccttcgccctaatccttcaaatccaaccttcaaactcaaccctcctcatcattttaggccttgcatccacccttatcggcggctgaggcgggttaaaccaaacacagctccgtaaaatccttgcatattcatcaatcgcccatctaggctgaataattcttgttttacaattttcaccctcaattacacttctcaccctcctcacctactttattataacattctcaacattccttatcttcaaactcaacaaatccacaaacattaacactcttgctatatcctggacgaaagctcctgccctcacagctctcacccccctcgtcctcctctcactagggggccttccccctcttacaggctttataccaaaatgactaattcttcaagaactagccaaacaagaccttgcccccgccgccaccctagcagccctctcagcccttctcagcctatatttttacctacgcctttcttatgcaataaccctcacgatttccccaaacaatcttacaagcacaactccctgacgcctaccttccacccagctaacttaccctctcgccacttcaaccgccataa

##CUDA

In [12]:
%%writefile CUDA.cu

#include <stdio.h>
#include <stdlib.h>
#include <stdint.h>
#include <string.h>
#include <cuda_runtime.h>

// Prefetch + page creation + memadvise
#define MAX_LENGTH (1 << 24)
typedef uint64_t bitvector;
__constant__ bitvector d_Eq[256];

__host__ __device__ int bit_vector_levenshtein(int query_length, const char *reference, int reference_length, const bitvector *Eq) {
    if (query_length > MAX_LENGTH || reference_length > MAX_LENGTH) {
        return -1;
    }

    bitvector Pv = ~0ULL;
    bitvector Mv = 0;
    bitvector Ph = 0;
    bitvector Mh = 0;
    bitvector Xv = 0;
    bitvector Xh = 0;
    bitvector Xp = 0;
    int score = query_length;

    for (int j = 0; j < reference_length; j++) {
        unsigned char c = reference[j];
        Xv = Eq[c] | Mv;

        Xh = ((~Xh & Xv) << 1) & Xp;

        // Explicit parentheses for clarity
        Xh = Xh | ((((Xv & Pv) + Pv) ^ Pv) | Xv | Mv);

        Ph = Mv | ~(Xh | Pv);
        Mh = Xh & Pv;
        Xp = Xv;

        if (Ph & (1ULL << (query_length - 1))) score++;
        if (Mh & (1ULL << (query_length - 1))) score--;

        Xv = (Ph << 1);
        Pv = (Mh << 1) | ~(Xh | Xv);
        Mv = Xh & Xv;

        if (score == 0) break;
    }

    return score;
}

__global__ void levenshtein_kernel(int query_length, const char *references, const int *reference_lengths, int *distances, int num_references) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    int stride = blockDim.x * gridDim.x;

    for (int i = idx; i < num_references; i += stride) {
        const char *reference = &references[i * MAX_LENGTH];
        int reference_length = reference_lengths[i];
        // Use the constant memory table on the device.
        distances[i] = bit_vector_levenshtein(query_length, reference, reference_length, d_Eq);
    }
}


char* read_file_into_string(const char* filename) {
    FILE* file = fopen(filename, "r");
    if (!file) {
        perror("Failed to open file");
        return NULL;
    }


    fseek(file, 0, SEEK_END);
    long file_size = ftell(file);
    fseek(file, 0, SEEK_SET);

    char* buffer = (char*)malloc(file_size + 1);
    if (!buffer) {
        perror("Failed to allocate memory");
        fclose(file);
        return NULL;
    }


    size_t bytes_read = fread(buffer, 1, file_size, file);
    if (bytes_read < file_size) {
        perror("Failed to read the file");
        free(buffer);
        fclose(file);
        return NULL;
    }

    buffer[bytes_read] = '\0';

    fclose(file);
    return buffer;
}


int main(){
    const char* query = read_file_into_string("input1.txt");
    int query_length = strlen(query);
    if (query_length > MAX_LENGTH) {
        printf("Query sequence too long!\n");
        return -1;
    }

    // Precompute Eq for the query and store it in host memory.
    bitvector h_Eq[256] = {0};
    for (int i = 0; i < query_length; i++) {
        h_Eq[(unsigned char)query[i]] |= (1ULL << i);
    }

    // Copy Eq to constant memory on the device.
    cudaMemcpyToSymbol(d_Eq, h_Eq, sizeof(bitvector) * 256);

    // Reference sequences to process
    const char* filenames[] = {"GQ167802.1.txt", "input1.txt"};
    int num_references = sizeof(filenames) / sizeof(filenames[0]);

    // Array to store the reference sequences
    const char* references_input[num_references];
    int reference_lengths[num_references];

    for (int i = 0; i < num_references; i++) {
        references_input[i] = read_file_into_string(filenames[i]);
        if (!references_input[i]) {
            fprintf(stderr, "Error reading file: %s\n", filenames[i]);

            // Free any previously allocated memory before exiting
            for (int j = 0; j < i; j++) {
                free((void*)references_input[j]);
            }
            return -1;
        }
        reference_lengths[i] = strlen(references_input[i]);
    }

    // Allocate Unified Memory
    char *references;
    int *d_reference_lengths, *d_distances;
    cudaMallocManaged(&references, num_references * MAX_LENGTH * sizeof(char));
    cudaMallocManaged(&d_reference_lengths, num_references * sizeof(int));
    cudaMallocManaged(&d_distances, num_references * sizeof(int));

    // Copy references and lengths to managed memory
    for (int i = 0; i < num_references; i++) {
        strncpy(&references[i * MAX_LENGTH], references_input[i], MAX_LENGTH);
        d_reference_lengths[i] = reference_lengths[i];
    }

    // Get GPU device ID
    int device = -1;
    cudaGetDevice(&device);

    // Memory advise
    cudaMemAdvise(references, num_references * MAX_LENGTH * sizeof(char), cudaMemAdviseSetReadMostly, device);
    cudaMemAdvise(d_reference_lengths, num_references * sizeof(int), cudaMemAdviseSetReadMostly, device);
    cudaMemAdvise(d_distances, num_references * sizeof(int), cudaMemAdviseSetPreferredLocation, device);

    // Prefetch data
    cudaMemPrefetchAsync(references, num_references * MAX_LENGTH * sizeof(char), device, NULL);
    cudaMemPrefetchAsync(d_reference_lengths, num_references * sizeof(int), device, NULL);
    cudaMemPrefetchAsync(d_distances, num_references * sizeof(int), device, NULL);

    // Kernel parameters
    int threadsPerBlock = 256;
    int blocksPerGrid = (num_references + threadsPerBlock - 1) / threadsPerBlock;

    // Number of times the program is executed
    const size_t loope = 10;

    printf("*** function = Levenshtein Distance\n");
    printf("numReferences = %d\n", num_references);
    printf("numBlocks = %d, numThreads = %d\n", blocksPerGrid, threadsPerBlock);

    // Execute kernel multiple times
    for (size_t i = 0; i < loope; i++) {
        levenshtein_kernel<<<blocksPerGrid, threadsPerBlock>>>(query_length, references, d_reference_lengths, d_distances, num_references);
    }

    // Synchronize device
    cudaDeviceSynchronize();

    // Prefetch distances back to CPU
    cudaMemPrefetchAsync(d_distances, num_references * sizeof(int), cudaCpuDeviceId, NULL);
    cudaDeviceSynchronize();

    // Error checking: compute expected distance on host using the host copy of Eq (h_Eq).
    size_t err_count = 0;
    for (int i = 0; i < num_references; i++) {
        int expected_distance = bit_vector_levenshtein(query_length, references_input[i], reference_lengths[i], h_Eq);
        if (d_distances[i] != expected_distance) {
            err_count++;
        }
    }
    printf("Error count (CUDA program): %zu\n", err_count);

    // Print results
    for (int i = 0; i < num_references; i++) {
        printf("Edit Distance between query \"%s\" and reference \"%s\" is: %d\n", query, references_input[i], d_distances[i]);
    }

    // Free memory
    cudaFree(references);
    cudaFree(d_reference_lengths);
    cudaFree(d_distances);

    for (int i = 0; i < num_references; i++) {
        free((void*)references_input[i]);
    }

    return 0;
}


Overwriting CUDA.cu


In [13]:
%%shell
nvcc -o CUDA CUDA.cu -arch=sm_75

In [14]:
%%shell
nvprof ./CUDA

==7290== NVPROF is profiling process 7290, command: ./CUDA
*** function = Levenshtein Distance
numReferences = 2
numBlocks = 1, numThreads = 256
Error count (CUDA program): 0
Edit Distance between query "tcg" and reference "ctttttggcttaggccttggcaccacaattacatttgctagctcccactgacttcttgcctgaataggccttgaaataaatacactagccattattcccctaatagctcaacatcatcacccccgcgctgtcgaagctacaactaaatattttttaacccaagctgctgctgcagccactcttctatttgcaagcattactaacgcctgactaacaggccaatgagaaattcaacaaattacacatcccctcccaaccacaataattacccttgcccttgccctcaaaatcggtcttgccccccttcatgcttgactccccgaagttttacaaggactagatcttaccacaggattaatcctctcaacctgacaaaaacttgctcccttcgccctaatccttcaaatccaaccttcaaactcaaccctcctcatcattttaggccttgcatccacccttatcggcggctgaggcgggttaaaccaaacacagctccgtaaaatccttgcatattcatcaatcgcccatctaggctgaataattcttgttttacaattttcaccctcaattacacttctcaccctcctcacctactttattataacattctcaacattccttatcttcaaactcaacaaatccacaaacattaacactcttgctatatcctggacgaaagctcctgccctcacagctctcacccccctcgtcctcctctcactagggggccttccccctcttacaggctttataccaaaatgactaattcttcaaga